In [3]:
# from comet_ml import Experiment
# experiment = Experiment(api_key="xktj4EX0zB8YcQ3BEaFwOQYpu")

In [4]:
import numpy as np
from keras.models import Sequential 
from keras.layers import Dense, Activation
from keras.regularizers import l2
from keras.utils import np_utils 

In [5]:
import os
import fnmatch

import pandas as pd

import sklearn.preprocessing as preprocessing
from sklearn.cross_validation import StratifiedKFold

C:\Users\Duo\Anaconda3\envs\fastai\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Load Data

In [6]:
train_path = "I:\\Honours-Project\\data\\sorted\\agg\\train\\"
train_csv = "I:\\Honours-Project\\data\\sorted\\agg\\train\\ids.csv"
test_path = "I:\\Honours-Project\\data\\sorted\\agg\\test\\"
test_csv = "I:\\Honours-Project\\data\\sorted\\agg\\test\\ids.csv"

dest_path = "I:\\Honours-Project\\data\\sorted\\agg\\"

csv = "I:\\Honours-Project\\data\\sorted\\agg\\ids.csv"

In [7]:
df = pd.read_csv(csv,index_col=0).sample(frac=1)

df['log_volume'] = df['volume'].apply(np.log)

x = df.drop(columns=['name','date','flow_rate','source','moments','inertia_tensor'])
# x = standardize = preprocessing.scale(x)

In [8]:
df.head()

,name,date,flow_rate,source,peak_val,area,volume,eccentricity,euler_number,extent,mean_intensity,moments,orientation,perimeter,inertia_tensor,cluster,num_peaks,log_volume
120,120,2017.11.22,7200,35.80 s.tifheightmap.mat,4.854192,2055,5440.452402,0.946396,1,0.368544,2.647422,[[2.05500000e+03 9.37710000e+04 5.11496900e+06...,0.886416,236.835570,[[292.97894613 278.03612482]\r\n [278.03612482...,2,1,8.601617
32,32,2017.11.17,3600,1.25 s.tifheightmap.mat,8.839536,1234,6886.017413,0.489669,1,0.624434,5.544297,[[1.24200000e+03 3.03310000e+04 8.91325000e+05...,1.298008,144.953319,[[ 95.96786714 7.6775466 ]\r\n [ 7.6775466 ...,0,1,8.837248
225,12_flipped,2017.11.09,3600,chip3_03heightmap.mat,12.532234,4756,38034.038483,0.756339,1,0.717995,7.997064,[[4.75600000e+03 2.31341000e+05 1.38869750e+07...,1.199091,292.693434,[[301.57312913 115.9663349 ]\r\n [115.9663349 ...,0,1,10.546237
385,84_flipped,2017.11.22,1800,44.60 s.tifheightmap.mat,21.427335,2675,23258.006678,0.921300,1,0.422992,8.694582,[[2.67500000e+03 1.34857000e+05 8.05168900e+06...,-0.940939,291.279221,[[ 295.92398288 -268.25541305]\r\n [-268.25541...,0,1,10.054405
47,47,2017.11.17,3600,26.45 s.tifheightmap.mat,4.989847,5936,16831.576998,0.972562,1,0.346972,2.835508,[[5.93600000e+03 6.72046000e+05 9.04600940e+07...,1.213680,497.244733,[[ 464.67299942 848.21577169]\r\n [ 848.21577...,2,1,9.731012


In [9]:
test_size = int(df.shape[0]/10)

x_test = np.array(x[:test_size])
y_test = df['flow_rate'][:test_size].values

x_train = np.array(x[test_size:])
y_train = df['flow_rate'][test_size:].values

In [10]:
print(np.bincount(y_train))
np.bincount(y_test)

[  0   0   0 ...   0   0 106]


array([ 0,  0,  0, ...,  0,  0, 10], dtype=int64)

## Run Logistic Regression

In [11]:
batch_size = 16
nb_classes = 3
nb_epoch = 20

In [12]:
input_dim = x_train.shape[1]

In [13]:
y_train[y_train == 1800] = 0
y_train[y_train == 3600] = 1
y_train[y_train == 7200] = 2

y_test[y_test == 1800] = 0
y_test[y_test == 3600] = 1
y_test[y_test == 7200] = 2

y_train = np_utils.to_categorical(y_train, nb_classes) 
y_test = np_utils.to_categorical(y_test, nb_classes)

In [14]:
lmda = 0.01

def build_logistic_model(input_dim, output_dim):
    model = Sequential()
    model.add(Dense(output_dim, input_dim=input_dim,
                    kernel_regularizer=l2(lmda),
                    activation='softmax'))

    return model

In [15]:
model = None
model = build_logistic_model(input_dim, nb_classes)

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 3)                 39        
Total params: 39
Trainable params: 39
Non-trainable params: 0
_________________________________________________________________


## Compile the model

In [17]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train,
                    batch_size=batch_size, epochs=nb_epoch,
                    verbose=1, validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)

Train on 362 samples, validate on 40 samples
Epoch 1/20
362/362 [==============================] - 4s 10ms/step - loss: 1.3396 - acc: 0.3674 - val_loss: 1.3562 - val_acc: 0.3250
Epoch 2/20
362/362 [==============================] - 0s 319us/step - loss: 1.2704 - acc: 0.3840 - val_loss: 1.3101 - val_acc: 0.3500
Epoch 3/20
362/362 [==============================] - 0s 449us/step - loss: 1.2233 - acc: 0.4006 - val_loss: 1.2745 - val_acc: 0.3500
Epoch 4/20
362/362 [==============================] - 0s 230us/step - loss: 1.1894 - acc: 0.3923 - val_loss: 1.2467 - val_acc: 0.3250
Epoch 5/20
362/362 [==============================] - 0s 276us/step - loss: 1.1643 - acc: 0.3812 - val_loss: 1.2251 - val_acc: 0.3250
Epoch 6/20
362/362 [==============================] - 0s 380us/step - loss: 1.1453 - acc: 0.3757 - val_loss: 1.2080 - val_acc: 0.3250
Epoch 7/20
362/362 [==============================] - 0s 319us/step - loss: 1.1315 - acc: 0.3867 - val_loss: 1.1938 - val_acc: 0.3250
Epoch 8/20
362/362

In [18]:
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 1.1210774540901185
Test accuracy: 0.425


## With Cross Validation

In [19]:
batch_size = 16
nb_classes = 3
nb_epoch = 20

In [20]:
def train_and_evaluate_model(model, x_train, y_train, x_test, y_test):
    model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(x_train, y_train,
                    batch_size=batch_size, epochs=nb_epoch,
                    verbose=1, validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test, verbose=0)
    return score[1]

In [34]:
n_folds = 10

labels = df['flow_rate'].values
data = x

labels[labels == 1800] = 0
labels[labels == 3600] = 1
labels[labels == 7200] = 2

labels = np_utils.to_categorical(labels, nb_classes) 

skf = StratifiedKFold(df['flow_rate'].values, n_folds=n_folds, shuffle=True)
avg_acc = 0

for i, (train, test) in enumerate(skf):
    print("Running Fold", i+1, "/", n_folds)
    model = None # Clearing the NN.
    model = build_logistic_model(input_dim, nb_classes)
    
    std = np.std(data[train],0)
    mean = np.mean(data[train],0)
    
    x_train = (data[train]-mean)/std
    x_test = (data[test] - mean)/std        
    
    avg_acc += train_and_evaluate_model(model, x_train, labels[train], x_test, labels[test])

Running Fold 1 / 10
Train on 360 samples, validate on 42 samples
Epoch 1/20
360/360 [==============================] - 1s 2ms/step - loss: 1.5124 - acc: 0.3389 - val_loss: 1.4932 - val_acc: 0.2857
Epoch 2/20
360/360 [==============================] - 0s 393us/step - loss: 1.4232 - acc: 0.3333 - val_loss: 1.3971 - val_acc: 0.2619
Epoch 3/20
360/360 [==============================] - 0s 371us/step - loss: 1.3596 - acc: 0.3306 - val_loss: 1.3300 - val_acc: 0.3095
Epoch 4/20
360/360 [==============================] - 0s 363us/step - loss: 1.3130 - acc: 0.3472 - val_loss: 1.2818 - val_acc: 0.3095
Epoch 5/20
360/360 [==============================] - 0s 396us/step - loss: 1.2782 - acc: 0.3528 - val_loss: 1.2447 - val_acc: 0.2857
Epoch 6/20
360/360 [==============================] - 0s 382us/step - loss: 1.2496 - acc: 0.3500 - val_loss: 1.2166 - val_acc: 0.3095
Epoch 7/20
360/360 [==============================] - 0s 342us/step - loss: 1.2273 - acc: 0.3444 - val_loss: 1.1938 - val_acc: 0.3095

361/361 [==============================] - 0s 248us/step - loss: 1.1162 - acc: 0.4072 - val_loss: 1.1136 - val_acc: 0.3902
Running Fold 4 / 10
Train on 361 samples, validate on 41 samples
Epoch 1/20
361/361 [==============================] - 1s 2ms/step - loss: 1.5514 - acc: 0.3130 - val_loss: 1.5405 - val_acc: 0.1463
Epoch 2/20
361/361 [==============================] - 0s 266us/step - loss: 1.4757 - acc: 0.3075 - val_loss: 1.4782 - val_acc: 0.1463
Epoch 3/20
361/361 [==============================] - 0s 251us/step - loss: 1.4151 - acc: 0.3186 - val_loss: 1.4289 - val_acc: 0.1220
Epoch 4/20
361/361 [==============================] - ETA: 0s - loss: 1.3126 - acc: 0.312 - 0s 299us/step - loss: 1.3669 - acc: 0.3075 - val_loss: 1.3900 - val_acc: 0.1463
Epoch 5/20
361/361 [==============================] - 0s 263us/step - loss: 1.3293 - acc: 0.2992 - val_loss: 1.3549 - val_acc: 0.1951
Epoch 6/20
361/361 [==============================] - 0s 272us/step - loss: 1.2968 - acc: 0.3186 - val_los

362/362 [==============================] - 0s 246us/step - loss: 1.1077 - acc: 0.4199 - val_loss: 1.1577 - val_acc: 0.3500
Epoch 20/20
362/362 [==============================] - 0s 271us/step - loss: 1.1031 - acc: 0.4227 - val_loss: 1.1514 - val_acc: 0.3500
Running Fold 7 / 10
Train on 363 samples, validate on 39 samples
Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 1.6831 - acc: 0.2893 - val_loss: 1.7026 - val_acc: 0.2308
Epoch 2/20
363/363 [==============================] - 0s 280us/step - loss: 1.5595 - acc: 0.3030 - val_loss: 1.6086 - val_acc: 0.2564
Epoch 3/20
363/363 [==============================] - 0s 259us/step - loss: 1.4593 - acc: 0.3030 - val_loss: 1.5327 - val_acc: 0.2564
Epoch 4/20
363/363 [==============================] - 0s 241us/step - loss: 1.3824 - acc: 0.3140 - val_loss: 1.4681 - val_acc: 0.2821
Epoch 5/20
363/363 [==============================] - 0s 265us/step - loss: 1.3222 - acc: 0.3223 - val_loss: 1.4159 - val_acc: 0.2821
Epoch 6/2

363/363 [==============================] - 0s 316us/step - loss: 1.0896 - acc: 0.4050 - val_loss: 1.1425 - val_acc: 0.4103
Epoch 19/20
363/363 [==============================] - 0s 313us/step - loss: 1.0858 - acc: 0.4187 - val_loss: 1.1392 - val_acc: 0.4103
Epoch 20/20
363/363 [==============================] - 0s 281us/step - loss: 1.0819 - acc: 0.4215 - val_loss: 1.1365 - val_acc: 0.4103
Running Fold 10 / 10
Train on 363 samples, validate on 39 samples
Epoch 1/20
363/363 [==============================] - 1s 3ms/step - loss: 1.5719 - acc: 0.3196 - val_loss: 1.4648 - val_acc: 0.2308
Epoch 2/20
363/363 [==============================] - 0s 288us/step - loss: 1.5127 - acc: 0.3223 - val_loss: 1.3901 - val_acc: 0.2564
Epoch 3/20
363/363 [==============================] - 0s 287us/step - loss: 1.4621 - acc: 0.3140 - val_loss: 1.3323 - val_acc: 0.2821
Epoch 4/20
363/363 [==============================] - 0s 316us/step - loss: 1.4191 - acc: 0.3306 - val_loss: 1.2867 - val_acc: 0.3333
Epoch 5

In [35]:
print("Average Score: ", avg_acc/n_folds)

Average Score:  0.3840741996038836
